In [1]:
!pip install optuna

You should consider upgrading via the 'c:\users\adhik\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.



  Using cached greenlet-1.1.2-cp39-cp39-win_amd64.whl (101 kB)
Using legacy 'setup.py install' for pyperclip, since package 'wheel' is not installed.
    Running setup.py install for pyperclip: started
    Running setup.py install for pyperclip: finished with status 'done'


In [2]:
!pip install xgboost

You should consider upgrading via the 'c:\users\adhik\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [15]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:
df = pd.read_csv('/02_Work/workspace/Datasets/Admission_Prediction.csv')

In [17]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


In [18]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [19]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].mean())

In [20]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [27]:
df.dtype()

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [21]:
x = df.drop(columns=['Serial No.', 'Chance of Admit'])
y = df['Chance of Admit']

In [22]:
x

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.000000,118.0,4.0,4.5,4.5,9.65,1
1,324.000000,107.0,4.0,4.0,4.5,8.87,1
2,316.558763,104.0,3.0,3.0,3.5,8.00,1
3,322.000000,110.0,3.0,3.5,2.5,8.67,1
4,314.000000,103.0,2.0,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332.000000,108.0,5.0,4.5,4.0,9.02,1
496,337.000000,117.0,5.0,5.0,5.0,9.87,1
497,330.000000,120.0,5.0,4.5,5.0,9.56,1
498,312.000000,103.0,4.0,4.0,5.0,8.43,0


In [23]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [11]:
from sklearn.metrics import mean_squared_error
def objective(trail, data = x, target= y):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=.10, random_state=30)
    params={
        'tree_method' :'gpu_hist',
        'lamda': trail.suggest_loguniform('lambda', 1e-4, 10.0),
        'alpha': trail.suggest_loguniform('alpha', 1e-4, 10.0),
        'colsample_bytree':trail.suggest_categorical('colsample_bytree', [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample':trail.suggest_categorical('subsample', [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate':trail.suggest_categorical('learning_rate', [.00001, .0003, .0008, .02, .01, 1, 10, 20]),
        'n_estimator':30000,
        'max_depth':trail.suggest_categorical('max_depth', [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
        'random_state':trail.suggest_categorical('random_state', [10, 20, 30, 2000, 3454, 243123]),
        'min_child_weight':trail.suggest_int('min_child_weight', 1, 200),
        'verbosity':3
    }

    xgb_reg_model = xgb.XGBRegressor(**params)
    xgb_reg_model.fit(train_x, train_y, eval_set=[(test_x, test_y)], verbose=True)
    pred_xgb = xgb_reg_model.predict(test_x)
    rmse = mean_squared_error(test_y, pred_xgb)
    return rmse
    

In [12]:
find_param = optuna.create_study(direction='minimize')

[I 2021-10-31 06:49:13,040] A new study created in memory with name: no-name-7ddda51c-09d6-45e1-b8a7-b0d77ed374dd


In [13]:
find_param.optimize(objective, n_trials=10)
find_param.best_trial.params

[06:49:16] DEBUG: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/tree/updater_gpu_hist.cu:892: [GPU Hist]: Configure
[06:49:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { lamda, n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[06:49:16] ======== Monitor: SketchContainer ========
[06:49:16] MakeCuts: 0.000776s, 1 calls @ 776us

[06:49:16] Prune: 0.000473s, 1 calls @ 473us

[06:49:16] ScanInput: 0.000448s, 1 calls @ 448us

[06:49:16] Unique: 0.000293s, 1 calls @ 293us

[0]	validation_0-rmse:2.06298
[1]	validation_0-rmse:17.81160
[2]	validation_0-rmse:154.13184
[3]	validation_0-rmse:1389.12781
[4]	validation_0-rmse:12486.39746
[5]	validation_0-rmse:111957.96094
[6]	validation_0-rmse:997468

[W 2021-10-31 06:49:17,023] Trial 0 failed because of the following error: ValueError("Input contains NaN, infinity or a value too large for dtype('float32').")
Traceback (most recent call last):
  File "C:\Users\adhik\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-11-921260ef908b>", line 21, in objective
    rmse = mean_squared_error(test_y, pred_xgb)
  File "C:\Users\adhik\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\adhik\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py", line 335, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "C:\Users\adhik\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py", line 90, in _check_reg_targets
    y_pred = ch

[06:49:17] EvalOneIter: 0.146066s, 100 calls @ 146066us

[06:49:17] GetGradient: 0.024715s, 100 calls @ 24715us

[06:49:17] PredictRaw: 0.450756s, 100 calls @ 450756us

[06:49:17] UpdateOneIter: 0.176301s, 100 calls @ 176301us

[06:49:17] ======== Monitor: GBTree ========
[06:49:17] BoostNewTrees: 0.15272s, 100 calls @ 152720us

[06:49:17] CommitModel: 0.005171s, 100 calls @ 5171us

[06:49:17] ======== Device 0 Memory Allocations:  ========
[06:49:17] Peak memory usage: 0MiB
[06:49:17] Number of allocations: 2311
[06:49:17] ======== Monitor: updater_gpu_hist ========
[06:49:17] InitData: 0.005589s, 100 calls @ 5589us

[06:49:17] InitDataOnce: 0.005561s, 1 calls @ 5561us

[06:49:17] Update: 0.151081s, 100 calls @ 151081us

[06:49:17] UpdatePredictionCache: 0.004983s, 100 calls @ 4983us

[06:49:17] ======== Monitor: gradient_based_sampler ========
[06:49:17] Sample: 0.005268s, 100 calls @ 5268us

[06:49:17] ======== Monitor: GPUHistMakerDevice0 ========
[06:49:17] AllReduce: 1.9e-05s, 11

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [24]:
!NVIDIA-SMI

Sun Oct 31 06:54:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 472.19       Driver Version: 472.19       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   48C    P8     6W /  N/A |    388MiB /  8192MiB |     32%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------